In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys
sys.path.append("../..") # relative path to module toolkit

In [3]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from toolkit import getLabelsFromDir, plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight 
from keras import optimizers
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
# setup
batch_size = 3
train_dir = "../../../images/images_species/train/"
val_dir = "../../../images/images_species/val/"
train_images = 12263
val_images = 3381
save_plot_name = "trainplot.png"
model_name = 'highest_val_acc.h5'

optimizer = optimizers.Adam(lr=0.0000001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [5]:
steps_per_epoch = int(train_images/batch_size) + 1
validation_steps = int(val_images/batch_size) + 1

In [6]:
### Model building ####

base_model = ResNet50(
            include_top=False,
            input_shape=(224, 224, 3),
            weights="imagenet")

#add a new dense layer to the end of the network inplace of the old layers
x = base_model.output
x = GlobalAveragePooling2D()(x)

# add the outplut layer
predictions = Dense(200, activation='softmax')(x)

# create new model composed of pre-trained network and new final layers
model = Model(input=base_model.input, output=predictions)

for layer in model.layers:
    layer.trainable = True

/home/herri/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/herri/.local/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  app.launch_new_instance()


In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [8]:

# compile model
model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])


In [9]:
labels = getLabelsFromDir(train_dir)

train_datagen = ImageDataGenerator(rescale=1./255.)
val_datagen = ImageDataGenerator(rescale=1./255.)

In [10]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(224, 224),
                                                    shuffle=True)
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(224, 224),
                                                    shuffle=True)

Found 12263 images belonging to 200 classes.
Found 3381 images belonging to 200 classes.


In [11]:
filepath = model_name

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

early_stop = EarlyStopping(monitor='val_acc', mode='max', patience=10)

callbacks = [checkpoint, early_stop]

In [12]:
the_class_weight = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)

In [13]:
#Train
history = model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=5000,
                    callbacks=callbacks,
                    validation_data=val_generator,
                    validation_steps=validation_steps,
                    class_weight=the_class_weight)

Epoch 1/5000
4088/4088 [==============================] - 490s 120ms/step - loss: 5.4834 - acc: 0.0060 - val_loss: 5.5953 - val_acc: 0.0056

Epoch 00001: val_acc improved from -inf to 0.00561, saving model to highest_val_acc.h5
Epoch 2/5000
4088/4088 [==============================] - 478s 117ms/step - loss: 5.4164 - acc: 0.0078 - val_loss: 5.5125 - val_acc: 0.0071

Epoch 00002: val_acc improved from 0.00561 to 0.00709, saving model to highest_val_acc.h5
Epoch 3/5000
4088/4088 [==============================] - 480s 117ms/step - loss: 5.3573 - acc: 0.0116 - val_loss: 5.4453 - val_acc: 0.0083

Epoch 00003: val_acc improved from 0.00709 to 0.00827, saving model to highest_val_acc.h5
Epoch 4/5000
4088/4088 [==============================] - 480s 117ms/step - loss: 5.2940 - acc: 0.0133 - val_loss: 5.3753 - val_acc: 0.0100

Epoch 00004: val_acc improved from 0.00827 to 0.01005, saving model to highest_val_acc.h5
Epoch 5/5000
4088/4088 [==============================] - 480s 117ms/step - los

4088/4088 [==============================] - 477s 117ms/step - loss: 3.9070 - acc: 0.2094 - val_loss: 3.6208 - val_acc: 0.2202

Epoch 00036: val_acc improved from 0.21602 to 0.22015, saving model to highest_val_acc.h5
Epoch 37/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 3.8804 - acc: 0.2155 - val_loss: 3.5607 - val_acc: 0.2264

Epoch 00037: val_acc improved from 0.22015 to 0.22636, saving model to highest_val_acc.h5
Epoch 38/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 3.8459 - acc: 0.2215 - val_loss: 3.5295 - val_acc: 0.2358

Epoch 00038: val_acc improved from 0.22636 to 0.23582, saving model to highest_val_acc.h5
Epoch 39/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 3.8040 - acc: 0.2261 - val_loss: 3.4897 - val_acc: 0.2414

Epoch 00039: val_acc improved from 0.23582 to 0.24143, saving model to highest_val_acc.h5
Epoch 40/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 3.7


Epoch 00072: val_acc did not improve from 0.39687
Epoch 73/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 2.8871 - acc: 0.3886 - val_loss: 2.5246 - val_acc: 0.3978

Epoch 00073: val_acc improved from 0.39687 to 0.39775, saving model to highest_val_acc.h5
Epoch 74/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 2.8725 - acc: 0.3960 - val_loss: 2.4753 - val_acc: 0.4122

Epoch 00074: val_acc improved from 0.39775 to 0.41223, saving model to highest_val_acc.h5
Epoch 75/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 2.8518 - acc: 0.4035 - val_loss: 2.4888 - val_acc: 0.4063

Epoch 00075: val_acc did not improve from 0.41223
Epoch 76/5000
4088/4088 [==============================] - 477s 117ms/step - loss: 2.8379 - acc: 0.4058 - val_loss: 2.4326 - val_acc: 0.4226

Epoch 00076: val_acc improved from 0.41223 to 0.42258, saving model to highest_val_acc.h5
Epoch 77/5000
4088/4088 [==============================] - 47

Epoch 111/5000
4088/4088 [==============================] - 478s 117ms/step - loss: 2.1244 - acc: 0.5577 - val_loss: 1.8070 - val_acc: 0.5405

Epoch 00111: val_acc did not improve from 0.54285
Epoch 112/5000
4088/4088 [==============================] - 496s 121ms/step - loss: 2.1154 - acc: 0.5616 - val_loss: 1.7881 - val_acc: 0.5387

Epoch 00112: val_acc did not improve from 0.54285
Epoch 113/5000
4088/4088 [==============================] - 1001s 245ms/step - loss: 2.0865 - acc: 0.5704 - val_loss: 1.7488 - val_acc: 0.5547

Epoch 00113: val_acc improved from 0.54285 to 0.55467, saving model to highest_val_acc.h5
Epoch 114/5000
4088/4088 [==============================] - 1153s 282ms/step - loss: 2.0674 - acc: 0.5687 - val_loss: 1.7559 - val_acc: 0.5517

Epoch 00114: val_acc did not improve from 0.55467
Epoch 115/5000
4088/4088 [==============================] - 1323s 324ms/step - loss: 2.0521 - acc: 0.5769 - val_loss: 1.7375 - val_acc: 0.5488

Epoch 00115: val_acc did not improve from 


Epoch 00149: val_acc did not improve from 0.64243
Epoch 150/5000
4088/4088 [==============================] - 1629s 398ms/step - loss: 1.5138 - acc: 0.6875 - val_loss: 1.3194 - val_acc: 0.6321

Epoch 00150: val_acc did not improve from 0.64243
Epoch 151/5000
4088/4088 [==============================] - 1594s 390ms/step - loss: 1.5083 - acc: 0.7009 - val_loss: 1.3203 - val_acc: 0.6389

Epoch 00151: val_acc did not improve from 0.64243
Epoch 152/5000
4088/4088 [==============================] - 1592s 389ms/step - loss: 1.4951 - acc: 0.6998 - val_loss: 1.3045 - val_acc: 0.6457

Epoch 00152: val_acc improved from 0.64243 to 0.64569, saving model to highest_val_acc.h5
Epoch 153/5000
4088/4088 [==============================] - 1597s 391ms/step - loss: 1.4836 - acc: 0.6993 - val_loss: 1.3027 - val_acc: 0.6442

Epoch 00153: val_acc did not improve from 0.64569
Epoch 154/5000
4088/4088 [==============================] - 1595s 390ms/step - loss: 1.4647 - acc: 0.7079 - val_loss: 1.2761 - val_ac

KeyboardInterrupt: 

In [14]:
plt.plot(history.history['acc'], label='training accuracy')
plt.plot(history.history['val_acc'], label='validation accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.savefig(save_plot_name, bbox_inches='tight')
plt.show()

NameError: name 'history' is not defined